In [1]:
import pandas as pd

In [2]:
df_genome_scores = pd.read_csv('genome-scores.csv')
df_genome_tags = pd.read_csv('genome-tags.csv')
df_link = pd.read_csv('links.csv')
df_movies = pd.read_csv('movies.csv')
df_ratings = pd.read_csv('ratings.csv')
df_tags = pd.read_csv('tags.csv')

In [12]:
df_genome_scores.iloc[44123]

movieId       40.00
tagId        132.00
relevance      0.07
Name: 44123, dtype: float64

In [19]:
df_movies[df_movies['movieId']==50]

,movieId,title,genres
49,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [7]:
# Pivot the genome scores to create a feature matrix
df_features = df_genome_scores.pivot(index='movieId', columns='tagId', values='relevance').fillna(0)
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity matrix for movies
cosine_sim = cosine_similarity(df_features)


In [10]:
len(df_genome_scores['movieId'].unique())

13816

In [19]:
import pickle
# Load the trained SVD model
with open('svd_model.pkl', 'rb') as f:
    svd = pickle.load(f)


In [1]:
def hybrid_recommendation(user_id, liked_movies, top_n=10):
    # Collaborative Filtering Recommendations
    user_rated_movies = df_ratings[df_ratings['userId'] == user_id]['movieId'].tolist()
    predictions = [svd.predict(user_id, mid) for mid in df_features.index if mid not in user_rated_movies]
    cf_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:top_n]
    cf_movie_ids = [pred.iid for pred in cf_recommendations]

    # Content-Based Filtering Recommendations
    liked_movie_ids = [df_movies[df_movies['title'] == movie]['movieId'].values[0] for movie in liked_movies]
    liked_movie_indices = [df_features.index.get_loc(mid) for mid in liked_movie_ids]
    content_sim_scores = cosine_sim[liked_movie_indices].mean(axis=0)
    content_movie_indices = content_sim_scores.argsort()[::-1][:top_n]
    
    content_movie_ids = [df_features.index[idx] for idx in content_movie_indices]

    # Combine Recommendations
    combined_movie_ids = list(set(cf_movie_ids + content_movie_ids))[:top_n]
    recommended_movies = [df_movies[df_movies['movieId'] == mid]['title'].values[0] for mid in combined_movie_ids]
    
    return recommended_movies

# Example usage
user_id = 1
liked_movies = ['Toy Story (1995)', 'Jumanji (1995)']
print(hybrid_recommendation(user_id, liked_movies, top_n=10))


NameError: name 'df_ratings' is not defined

In [1]:
import secrets
secret_key = secrets.token_hex(24)
print(secret_key)

72d373c2256e074a908498b1c7bbd85f5a5a603ff233d451


In [10]:
def split_title_and_year(title):
    if ' (' in title and title.endswith(')'):
        parts = title.rsplit(' (', 1)
        year = parts[-1][:-1]  # Remove the closing parenthesis
        if year.isdigit():
            title = parts[0]
            return title.strip(), int(year)
    return title, None


# Apply the function to the title column and create new columns
df_movies[['title', 'year']] = df_movies['title'].apply(lambda x: pd.Series(split_title_and_year(x)))


In [16]:
df_movies[df_movies['year']>1999]

,movieId,title,genres,year
2677,2769,"Yards, The",Crime|Drama,2000.0
3084,3177,Next Friday,Comedy,2000.0
3097,3190,Supernova,Adventure|Sci-Fi|Thriller,2000.0
3132,3225,Down to You,Comedy|Romance,2000.0
3135,3228,Wirey Spindell,Comedy,2000.0
...,...,...,...,...
62417,209155,Santosh Subramaniam,Action|Comedy|Romance,2008.0
62418,209157,We,Drama,2018.0
62419,209159,Window of the Soul,Documentary,2001.0
62420,209163,Bad Poems,Comedy|Drama,2018.0


In [18]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate

# Load your dataset (replace 'ratings.csv' and 'movies.csv' with your actual dataset files)
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

# Function to split title and year
def split_title_and_year(title):
    if ' (' in title and title.endswith(')'):
        parts = title.rsplit(' (', 1)
        year = parts[-1][:-1]  # Remove the closing parenthesis
        if year.isdigit():
            return title, int(year)
    return title, None

# Apply the function to the title column and create new columns
movies[['title', 'year']] = movies['title'].apply(lambda x: pd.Series(split_title_and_year(x)))

# Filter movies from the year 2000 and onwards
# movies_filtered = movies[movies['year'] >= 2000]

# # Merge the ratings with the filtered movies to get only relevant ratings
# filtered_ratings = pd.merge(ratings, movies_filtered[['movieId']], on='movieId')

# # Prepare data for the Surprise library
# reader = Reader(rating_scale=(0.5, 5.0))
# data = Dataset.load_from_df(filtered_ratings[['userId', 'movieId', 'rating']], reader)

# # Train the SVD model
# svd = SVD()

# # Evaluate the model using cross-validation
# cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# # Fit the model on the whole dataset
# trainset = data.build_full_trainset()
# svd.fit(trainset)

# # Save the trained model (optional)
# import pickle
# with open('svd_model.pkl', 'wb') as f:
#     pickle.dump(svd, f)

# # Example of making a prediction
# user_id = 1  # replace with actual user_id
# movie_id = 1  # replace with actual movie_id
# prediction = svd.predict(user_id, movie_id)
# print(prediction)


In [22]:
df_movies_filtered = movies[movies['year']>1999]

In [25]:
df_movies_filtered.drop('year',axis=1).to_pickle('movies_2000.pkl')

In [23]:
movies_filtered.

In [28]:
df_genome_scores.merge(movies_filtered['movieId'],on='movieId',how='right').dropna().to_pickle('filtered_genome.pkl')

In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle('movies_2000.pkl')

In [3]:
df

,movieId,title,genres,year
2677,2769,"Yards, The",Crime|Drama,2000.0
3084,3177,Next Friday,Comedy,2000.0
3097,3190,Supernova,Adventure|Sci-Fi|Thriller,2000.0
3132,3225,Down to You,Comedy|Romance,2000.0
3135,3228,Wirey Spindell,Comedy,2000.0
...,...,...,...,...
62417,209155,Santosh Subramaniam,Action|Comedy|Romance,2008.0
62418,209157,We,Drama,2018.0
62419,209159,Window of the Soul,Documentary,2001.0
62420,209163,Bad Poems,Comedy|Drama,2018.0


In [13]:
df_links = pd.read_csv('links.csv').merge(df['movieId'],on='movieId',how='right')

In [16]:
df_links.to_pickle('links_2000.pkl')

In [29]:
df = df_links.merge(movies,on='movieId')

In [32]:
df[df['title']=='The Blue Planet (2001)']

,movieId,imdbId,tmdbId,title,genres,year
14619,142115,296310,200813.0,The Blue Planet (2001),Documentary,2001.0
